In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import pyzbar.pyzbar as pyzbar

In [ ]:
img = cv2.imread('qr/0007.bmp')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
barcodes=pyzbar.decode(img)
print(barcodes)
plt.imshow(img, cmap='gray')

In [ ]:
img = 255 - img
blur = cv2.medianBlur(img, 1)
plt.imshow(blur, cmap='gray')

In [ ]:
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 15))
morphed = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
plt.imshow(morphed, cmap='gray')

In [ ]:
output = 255 - cv2.subtract(img, morphed)
barcodes=pyzbar.decode(output)
print(barcodes)
plt.imshow(output, cmap='gray')

In [ ]:
# kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 10))
# morph_img = cv2.morphologyEx(output, cv2.MORPH_OPEN, kernel)

# test = cv2.adaptiveThreshold(morph_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 25, 2)
# plt.imshow(morph_img, cmap='gray')

In [ ]:
# canny = cv2.Canny(test, 40, 150)

# contour_img = cv2.cvtColor(output, cv2.COLOR_GRAY2BGR)
# contours, _ = cv2.findContours(canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# contour_img = cv2.drawContours(contour_img, contours, -1, (255, 0, 0), 1)

# plt.imshow(contour_img)

In [ ]:
# max_contour ,max_area = 0, 0
# for contour in contours:
#     tmp = cv2.contourArea(contour)
#     if tmp > max_area:
#         max_contour, max_area = contour, tmp
# print(max_contour.shape)
# max_contour = max_contour[::-1, :, :]

# # robustness：如果不是4怎么办   0.02来自于网络
# poly_contour = cv2.approxPolyDP(max_contour, 0.02 * cv2.arcLength(max_contour, True), True)

# points = poly_contour.squeeze()
# # assert(len(poly_contour) == 4)
# print(poly_contour)

In [ ]:
# c = np.cross(points[1]-points[0], points[2]-points[1])
# if c > 0:   # 修正为右旋标价
#     points = points[::-1, :]

# len1 = np.linalg.norm(points[0] - points[1])
# len2 = np.linalg.norm(points[1] - points[2])
# if len1 >= len2:    # 短边优先
#     points = np.roll(points, 1, axis=0)

# if points[0][0] > points[2][0]: # y值小的优先
#     points = np.roll(points, 2, axis=0)

# outer_x, outer_y = 10, 10
# shape_x, shape_y = 105, 105

# N = np.array([[outer_x, outer_y], [outer_x, shape_y - outer_y], 
#             [shape_x - outer_x, shape_y - outer_y], [shape_x - outer_x, outer_y]])
# mat = cv2.getPerspectiveTransform(points.astype(np.float32), N.astype(np.float32))
# output1 = cv2.warpPerspective(output, mat, (shape_x, shape_y), borderValue=255)
# plt.imshow(output1, cmap='gray')

In [ ]:
# output = output1

laplacian = cv2.Laplacian(output, -1)
new_img =cv2.subtract(output, laplacian)
barcodes=pyzbar.decode(new_img)
print(barcodes)
plt.imshow(new_img, cmap='gray')

output = new_img

In [ ]:
test = cv2.adaptiveThreshold(new_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 21, 2)
barcodes=pyzbar.decode(test)
print(barcodes)
plt.imshow(test, cmap='gray')

In [ ]:
for i in range(127, 255):  #粗暴的阈值处理（可能先通过某种算法调整对比度）
    _, output2 = cv2.threshold(new_img, i, 255, cv2.THRESH_BINARY)
    barcodes=pyzbar.decode(output2)
    if len(barcodes) != 0:
        break
# _, output2 = cv2.threshold(new_img, 0, 255, cv2.THRESH_OTSU)

print(i, barcodes)
plt.imshow(output2, cmap='gray')

### 一个简单的汇总

In [ ]:
def fix_img(old_img):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 10))
    morph_img = cv2.morphologyEx(old_img, cv2.MORPH_OPEN, kernel)

    test = cv2.adaptiveThreshold(morph_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 25, 2)
    canny = cv2.Canny(test, 40, 150)

    max_contour ,max_area = 0, 0
    for contour in contours:
        tmp = cv2.contourArea(contour)
        if tmp > max_area:
            max_contour, max_area = contour, tmp
    
    max_contour = max_contour[::-1, :, :]
    poly_contour = cv2.approxPolyDP(max_contour, 0.02 * cv2.arcLength(max_contour, True), True)
    points = poly_contour.squeeze()
    if len(poly_contour) != 4:
        return None

    c = np.cross(points[1]-points[0], points[2]-points[1])
    if c > 0:   # 修正为右旋标价
        points = points[::-1, :]

    len1 = np.linalg.norm(points[0] - points[1])
    len2 = np.linalg.norm(points[1] - points[2])
    if len1 >= len2:    # 短边优先
        points = np.roll(points, 1, axis=0)

    if points[0][0] > points[2][0]: # y值小的优先
        points = np.roll(points, 2, axis=0)

    outer_x, outer_y = 10, 10
    shape_x, shape_y = 105, 105

    N = np.array([[outer_x, outer_y], [outer_x, shape_y - outer_y], 
                [shape_x - outer_x, shape_y - outer_y], [shape_x - outer_x, outer_y]])
    mat = cv2.getPerspectiveTransform(points.astype(np.float32), N.astype(np.float32))
    return cv2.warpPerspective(output, mat, (shape_x, shape_y), borderValue=255)


In [ ]:
def recog_qr(img):
    barcodes=pyzbar.decode(img)
    if len(barcodes) > 0:
        return 0, barcodes[0].data.decode('utf-8')
    
    img = 255 - img
    blur = cv2.medianBlur(img, 5)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (15, 15))
    morphed = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    output = 255 - cv2.subtract(img, morphed)
    barcodes=pyzbar.decode(output)
    if len(barcodes) > 0:
        return 1, barcodes[0].data.decode('utf-8')
    

    for att in range(3):
        laplacian = cv2.Laplacian(output, -1)
        new_img =cv2.subtract(output, laplacian)
        barcodes=pyzbar.decode(new_img)
        if len(barcodes) > 0:
            return 2, barcodes[0].data.decode('utf-8')
        
        test = cv2.adaptiveThreshold(new_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 15, 2)
        barcodes=pyzbar.decode(test)
        if len(barcodes) > 0:
            return 3, barcodes[0].data.decode('utf-8')
        test = cv2.adaptiveThreshold(new_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 21, 2)
        barcodes=pyzbar.decode(test)
        if len(barcodes) > 0:
            return 3, barcodes[0].data.decode('utf-8')
        
        for i in range(127, 256):  #粗暴的阈值处理（可能先通过某种算法调整对比度）
            _, output2 = cv2.threshold(new_img, i, 255, cv2.THRESH_BINARY)
            barcodes=pyzbar.decode(output2)
            if len(barcodes) != 0:
                return 4, barcodes[0].data.decode('utf-8')
        else:
            # output = fix_img(new_img)
            # if output is None:
            return -1, ''
    return -1, ''
    

In [ ]:
input_path = 'qr'
output_file = 'predictions.txt'
counter = [0, 0, 0, 0, 0]

with open(output_file, 'w') as f:
    for file_name in tqdm(os.listdir(input_path)):
        img = cv2.imread(os.path.join(input_path, file_name))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        status, result = recog_qr(img)
        f.write('{} {}\n'.format(file_name, result))
        if status != -1:
            counter[status] += 1

total = sum(counter)
print()
print('Sum*', 'Base', 'T-hat', 'lapla', 'athrs', 'thrs', sep='\t')
print(total, *counter, sep='\t')